In [21]:
from astropy.io import fits
from pygsm import GSMObserver
import numpy as np
from scipy import interpolate
from datetime import datetime
from astropy.time import Time
import matplotlib.pyplot as plt
import healpy as hp
import time

In [48]:
hera_beam_file = '/Users/tyler/Desktop/Research/Tsys/data/HERA_beam_nic.hmap'

df = 1.5625  # 100 MHz / 64 averaged channels
freqs = np.arange(100.0 + df / 2.0, 200.0, df)
hours = np.arange(0.0, 24.0, .5)
lsts = np.zeros_like(hours)
pols = ['X', 'Y']  # Only have X beam, but try rotating 90 degrees for Y
HERA_Tsky = np.zeros((len(pols), freqs.shape[0], lsts.shape[0]))

In [23]:
# Read in HERA beam data, just use full sky for paper
hera_beam = {}
# Only have X right now, will rotate later
hera_im = fits.getdata(hera_beam_file, extname='BEAM_{0}'.format('X'))
nside = hp.npix2nside(hera_im.shape[0])
temp_f = fits.getdata(hera_beam_file, extname='FREQS_{0}'.format('X'))
# Interpolate to the desired frequencies
func = interpolate.interp1d(temp_f, hera_im, kind='cubic', axis=1)

for pol in pols:
    hera_beam[pol] = func(freqs)

In [24]:
# Set up the observer
(latitude, longitude, elevation) = ('-30.7224', '21.4278', 1100)
ov = GSMObserver()
ov.lon = longitude
ov.lat = latitude
ov.elev = elevation

In [25]:
proj_sky = hp.projector.OrthographicProj(rot=[0,0,0], half_sky=True, xsize=400)
f = lambda x,y,z: hp.pixelfunc.vec2pix(nside,x,y,z,nest=False)

In [32]:
vals = []

for poli, pol in enumerate(pols[:1]):
    pol_ang = 90 * (1 - poli)  # Extra rotation for X
    proj_beam = hp.projector.OrthographicProj(rot=[pol_ang,90], half_sky=True, xsize=400)
    for fi, freq in enumerate(freqs[:1]):
        
        print 'Forming HERA Tsky for frequency ' + str(freq) + ' MHz.'
    
        #deg = 8

        #smoothed_beam = hp.sphtfunc.smoothing(hera_beam[pol][:, fi], fwhm=0.017*deg)
        hbeam1 = proj_beam.projmap(hera_beam[pol][:, fi], f)
        hbeam1[np.isinf(hbeam1)] = np.nan

        for ti, t in enumerate(hours[:]):
            dt = datetime(2013, 1, 1, np.int(t), np.int(60.0 * (t - np.floor(t))),
                          np.int(60.0 * (60.0 * t - np.floor(t * 60.0))))
            lsts[ti] = Time(dt).sidereal_time('apparent', longitude).hour
            ov.date = dt
            observed_sky = ov.generate(freq)
            nside_sky = hp.pixelfunc.npix2nside(hp.pixelfunc.get_map_size(observed_sky))
            f_sky = lambda x,y,z: hp.pixelfunc.vec2pix(nside_sky,x,y,z, nest=False)
            sky = proj_sky.projmap(observed_sky, f_sky)
            sky[np.isinf(sky)] = np.nan
            HERA_Tsky[poli, fi, ti] = np.nanmean(hbeam1 * sky) / np.nanmean(hbeam1)
            vals.append(np.nanmean(hbeam1 * sky) / np.nanmean(hbeam1))

Forming HERA Tsky for frequency 100.78125 MHz.
